In optimization attempt 1, we are just going to rebin the categorical data into less bins and see if that makes a big enough difference to get us above 75%

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
attr = application_df['APPLICATION_TYPE'].value_counts() # attr stands for application types to replace

application_types_to_replace = attr[attr.lt(2000)].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     7262
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x: x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
ctr = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = ctr[ctr.lt(2000)].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [13]:
# Checking for Affiliation types value counts
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [16]:
# Choose a cutoff value and create a list of affiliations to be replaced
# use the variable name `affiliations_to_replace`
atr = application_df['AFFILIATION'].value_counts()
affiliations_to_replace = atr[atr.lt(2000)].index

# Replace in dataframe
for aff in affiliations_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [15]:
# Checking for USE_CASE types value counts
application_df['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [19]:
# Choose a cutoff value and create a list of use cases to be replaced
# use the variable name `use_cases_to_replace`
uctr = application_df['USE_CASE'].value_counts()
use_cases_to_replace = uctr[uctr.lt(6000)].index

# Replace in dataframe
for use in use_cases_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(use,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

Preservation    28095
Other            6204
Name: USE_CASE, dtype: int64

In [18]:
# Checking for ORGANIZATION types value counts
application_df['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [20]:
# Choose a cutoff value and create a list of organizations to be replaced
# use the variable name `organizations_to_replace`
otr = application_df['ORGANIZATION'].value_counts()
organizations_to_replace = otr[otr.lt(2000)].index

# Replace in dataframe
for org in organizations_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(org,"Other")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [21]:
# Checking for income amount value counts
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [22]:
# Choose a cutoff value and create a list of incomes to be replaced
# use the variable name `incomes_to_replace`
itr = application_df['INCOME_AMT'].value_counts()
incomes_to_replace = itr[itr.lt(5000)].index

# Replace in dataframe
for income in incomes_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(income,"Other")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

0        24388
Other     9911
Name: INCOME_AMT, dtype: int64

In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_categorical = application_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT']]
application_df_numerical = application_df[['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL']]

application_df_categorical = pd.get_dummies(application_df_categorical)

application_df_2 = pd.concat((application_df_categorical, application_df_numerical), axis=1)
application_df_2.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_Other,USE_CASE_Other,...,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_0,INCOME_AMT_Other,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1,0,0,1,0,1,0,0,0,1,...,1,0,0,1,0,1,0,1,5000,1
1,0,1,0,1,0,0,0,1,0,0,...,0,1,0,1,0,0,1,1,108590,1
2,1,0,1,0,0,0,0,0,1,1,...,1,0,0,1,0,1,0,1,5000,0
3,0,1,1,0,0,0,0,1,0,0,...,0,0,1,1,0,0,1,1,6692,1
4,0,1,0,1,0,1,0,0,0,1,...,0,0,1,1,0,0,1,1,142590,1


In [24]:
# Split our preprocessed data into our features and target arrays
y = application_df_2['IS_SUCCESSFUL']

application_df_2_copy = application_df_2.copy()
X = application_df_2_copy.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X.shape[1]
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 168       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 219 (876.00 Byte)
Trainable params: 219 (876.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 6ms/step - loss: 0.6215 - accuracy: 0.6677
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5945 - accuracy: 0.6987
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5896 - accuracy: 0.7028
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5878 - accuracy: 0.7043
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5867 - accuracy: 0.7044
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5855 - accuracy: 0.7054
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5847 - accuracy: 0.7065
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5841 - accuracy: 0.7052
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5839 - accuracy: 0.7049
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5837 - accura

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5916 - accuracy: 0.7046 - 966ms/epoch - 4ms/step
Loss: 0.591606080532074, Accuracy: 0.7046064138412476


In [31]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimized1.h5')

from google.colab import files
files.download("AlphabetSoupCharity_Optimized1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>